In [43]:
# THE OG

import requests
import json
import pandas as pd
import numpy as np
import random

In [44]:
# ESPN Points DB
# Nicholas Culmone
# Dec 6th 2022
# Focus: Getting the entire ESPN player database and the fantasy points generated by all active players
#


# first api call to player database
url = "https://statsapi.web.nhl.com/api/v1/teams/?expand=team.roster"
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
  'Accept': '*/*',
  'Accept-Language': 'en-CA,en-US;q=0.7,en;q=0.3',
  'Referer': 'http://www.nhl.com/',
  'Origin': 'http://www.nhl.com',
  'Connection': 'keep-alive',
  'TE': 'Trailers'
}
r = requests.get(url, headers=headers)
teams = r.json()
df = pd.json_normalize(teams)

# defining player lists
nhlfwds = []
nhldef = []
nhlgoal = []

#0,1
#len(df.teams[0])
for i in range(len(df.teams[0])):
    
    # current team's abbv
    abbv = df.teams[0][i]['abbreviation']
    print(abbv)
    
    # current team's roster
    roster = df.teams[0][i]['roster']['roster']
    
    for j in range(len(roster)):
        
        # important variable decl.
        pers = roster[j]['person']
        persid = pers['id']
        persname = pers['fullName']
        perspos = roster[j]['position']['code']
        
        # api call for getting selected player's stats
        # todo, make this call less often hopefully...
        url = "https://statsapi.web.nhl.com/api/v1/people/" + str(persid) + "/stats?stats=statsSingleSeason&season=20222023"
        headers = {
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
          'Accept': '*/*',
          'Accept-Language': 'en-CA,en-US;q=0.7,en;q=0.3',
          'Referer': 'http://www.nhl.com/',
          'Origin': 'http://www.nhl.com',
          'Connection': 'keep-alive',
          'TE': 'Trailers'
        }
        r = requests.get(url, headers=headers)
        stats = r.json()
        pdf = pd.json_normalize(stats)
        
        # player could be inactive/not played a game on a roster
        # try/catch to avoid these types of players
        try:
            # forwards
            if perspos == 'L' or perspos == 'R' or perspos == 'C':
                stdb = pdf.stats[0][0]['splits'][0]['stat']
                fpts = float(round(float(2 * stdb['goals'] + stdb['assists'] + stdb['powerPlayPoints'] + 2 * stdb['shortHandedPoints']) - .25 * float(stdb['penaltyMinutes']) + .2 * float(stdb['blocked']), 2))
                #print("F -", fpts, "||", stdb['games'], stdb['goals'], stdb['assists'], st['penaltyMinutes'], stdb['powerPlayPoints'], stdb['shortHandedPoints'], stdb['blocked'])
                
                ele = [persid, persname, abbv, perspos, fpts, stdb['games'], stdb['goals'], stdb['assists'], stdb['penaltyMinutes'], stdb['powerPlayPoints'], stdb['shortHandedPoints'], stdb['blocked']]
                print(ele)
                
                nhlfwds.append(ele)
            
            # defense
            elif perspos == 'D':
                stdb = pdf.stats[0][0]['splits'][0]['stat']
                fpts = float(round(float(2 * stdb['goals'] + stdb['assists'] + stdb['powerPlayPoints'] + 2 * stdb['shortHandedPoints']) - .25 * float(stdb['penaltyMinutes']) + .2 * float(stdb['blocked']), 2))
                
                ele = [persid, persname, abbv, perspos, fpts, stdb['games'], stdb['goals'], stdb['assists'], stdb['penaltyMinutes'], stdb['powerPlayPoints'], stdb['shortHandedPoints'], stdb['blocked']]
                print(ele)
                
                nhldef.append(ele)
            
            #goalies
            elif perspos == 'G':
                print(stdb = pdf.stats[0][0]['splits'][0]['stat'])
                
                
        except:
            print("Whoops")

# dataframe for forwards
fwddf = pd.DataFrame(nhlfwds, columns=['id', 'name', 'team', 'pos', 'fpts', 'gp', 'g', 'a', 'pim', 'ppp', 'shp', 'blk'])
#display(fwddf)
display(fwddf.sort_values(by=['fpts'], ascending=False))

# dataframe for defensemen
defdf = pd.DataFrame(nhldef, columns=['id', 'name', 'team', 'pos', 'fpts', 'gp', 'g', 'a', 'pim', 'ppp', 'shp', 'blk'])
#display(defdf)
display(defdf.sort_values(by=['fpts'], ascending=False))

NJD
Whoops
[8476292, 'Ondrej Palat', 'NJD', 'L', 6.8, 6, 3, 0, '0', 0, 0, 4]
[8474090, 'Brendan Smith', 'NJD', 'D', -0.5, 32, 0, 2, '38', 0, 0, 35]
[8475193, 'Tomas Tatar', 'NJD', 'L', 25.5, 33, 8, 9, '14', 0, 0, 20]
[8475287, 'Erik Haula', 'NJD', 'L', 17.25, 33, 2, 12, '27', 3, 0, 25]
[8476462, 'Dougie Hamilton', 'NJD', 'D', 46.1, 33, 7, 19, '22', 12, 0, 33]
[8476923, 'Damon Severson', 'NJD', 'D', 11.4, 32, 2, 4, '24', 2, 0, 37]
[8477341, 'Andreas Johnsson', 'NJD', 'L', 0.0, 2, 0, 0, '0', 0, 0, 0]
[8477425, 'Miles Wood', 'NJD', 'L', 12.85, 32, 7, 7, '39', 0, 0, 8]
[8477435, 'Ryan Graves', 'NJD', 'D', 22.6, 33, 3, 6, '8', 0, 1, 53]
Whoops
[8478399, 'Jonas Siegenthaler', 'NJD', 'D', 20.3, 33, 1, 11, '22', 1, 0, 59]
Whoops
[8478507, 'John Marino', 'NJD', 'D', 18.1, 32, 3, 6, '10', 0, 0, 43]
[8479407, 'Jesper Bratt', 'NJD', 'L', 59.2, 33, 12, 21, '4', 13, 0, 11]
[8479414, 'Nathan Bastian', 'NJD', 'R', 10.85, 21, 3, 5, '15', 0, 0, 18]
[8479415, 'Michael McLeod', 'NJD', 'C', 12.7, 33, 3, 11

,id,name,team,pos,fpts,gp,g,a,pim,ppp,shp,blk
292,8478402,Connor McDavid,EDM,C,129.20,34,29,36,16,34,1,16
290,8477934,Leon Draisaitl,EDM,C,104.60,34,21,34,16,26,1,23
92,8479420,Tage Thompson,BUF,C,97.50,32,26,24,18,22,1,10
338,8480027,Jason Robertson,DAL,L,87.80,34,24,24,8,17,0,4
79,8477956,David Pastrnak,BOS,R,86.30,32,22,23,18,22,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...
346,8477962,Brendan Lemieux,LAK,L,-2.60,15,0,3,28,0,0,7
174,8474034,Pat Maroon,TBL,L,-4.65,32,1,4,49,1,0,3
368,8479981,Jonah Gadjovich,SJS,L,-5.35,19,1,1,35,0,0,2
420,8475178,Zack Kassian,ARI,R,-5.90,15,1,0,34,0,0,3


,id,name,team,pos,fpts,gp,g,a,pim,ppp,shp,blk
200,8474578,Erik Karlsson,SJS,D,76.50,35,13,33,10,13,0,35
48,8480839,Rasmus Dahlin,BUF,D,68.20,31,10,27,40,20,0,56
16,8479323,Adam Fox,NYR,D,63.60,35,7,28,12,13,1,48
227,8477504,Josh Morrissey,WPG,D,62.25,33,6,33,23,14,0,45
158,8480069,Cale Makar,COL,D,56.70,31,7,22,22,17,0,46
...,...,...,...,...,...,...,...,...,...,...,...,...
189,8477810,Joel Hanley,DAL,D,-1.45,8,0,0,13,0,0,9
126,8474697,Mark Borowiecki,NSH,D,-2.80,4,0,0,12,0,0,1
140,8478502,Dennis Gilbert,CGY,D,-2.80,8,0,1,20,0,0,6
153,8477073,Kurtis MacDermid,COL,D,-2.80,10,0,1,16,0,0,1


In [41]:
# Gets the CURRENT GAMES OF THAT DAY

url = "https://livescore6.p.rapidapi.com/matches/v2/list-by-league"
querystring = {"Category":"hockey","Ccd":"nhl","Timezone":"-7"}

headers = {
	"X-RapidAPI-Key": "c055849d88mshdde5983aeb41817p11914djsn741b0784c921",
	"X-RapidAPI-Host": "livescore6.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

{'Stages': [{'Sid': '11051',
   'Snm': 'NHL: Regular season',
   'Scd': 'nhl-regular-season',
   'Cid': '345',
   'Cnm': 'NHL',
   'Csnm': 'NHL',
   'Ccd': 'nhl',
   'Scu': 0,
   'Sds': 'NHL: Regular season',
   'Ccdiso': 'USA',
   'Chi': 0,
   'Shi': 0,
   'Sdn': 'NHL: Regular season',
   'Events': [{'Eid': '736894',
     'Pids': {'1': '3941486', '8': '736894', '12': 'SBTE_28116715'},
     'Sids': {'1': '880281', '8': '11051', '12': 'SBTC3_42133'},
     'Tr1': '4',
     'Tr2': '1',
     'Tr1OR': '4',
     'Tr2OR': '1',
     'Tr1Pe1': '1',
     'Tr2Pe1': '0',
     'Tr1Pe2': '2',
     'Tr2Pe2': '0',
     'Tr1Pe3': '1',
     'Tr2Pe3': '1',
     'T1': [{'Nm': 'Carolina Hurricanes',
       'ID': '1630',
       'Img': 'enet/282.png',
       'Abr': 'CAR',
       'tbd': 0,
       'Gd': 1,
       'Pids': {'1': ['282'], '8': ['1630'], '12': ['SBTP_6948']},
       'CoNm': 'USA',
       'CoId': 'USA',
       'HasVideo': False}],
     'T2': [{'Nm': 'New Jersey Devils',
       'ID': '1655',
       

In [59]:
url = "https://statsapi.web.nhl.com/api/v1/schedule"

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
  'Accept': '*/*',
  'Accept-Language': 'en-CA,en-US;q=0.7,en;q=0.3',
  'Referer': 'http://www.nhl.com/',
  'Origin': 'http://www.nhl.com',
  'Connection': 'keep-alive',
  'TE': 'Trailers'
}

r = requests.get(url, headers=headers)
games = r.json()
df = pd.json_normalize(games)

totalGames = df.totalGames[0]

df2 = pd.json_normalize(df.dates[0][0])
df3 = pd.json_normalize(df2.games[0])

display(df3)

dfSkater = pd.DataFrame(columns=['Name', 'ID','G','A','PIM','PPP','SHP','BLK','FPTS'])
dfGoalie = pd.DataFrame(columns=['Name', 'ID','W','GA','SV','SO','FPTS'])



for i in range(totalGames):
    print(df3.link[i])
    url2 = "https://statsapi.web.nhl.com" + df3.link[i]
    r2 = requests.get(url2, headers=headers)
    curGame = r2.json()
    numPlays = len(curGame["liveData"]["plays"]["allPlays"])
    gameArray = []
    
    for j in range(numPlays):
        curPlay = curGame["liveData"]["plays"]["allPlays"][j]
        eventType = curPlay["result"]["eventTypeId"]
        print(eventType)
        s
        if(eventType == 'BLOCKED_SHOT'):
            #display(str(curPlay["players"][0]["player"]["id"]) + " " + curPlay["players"][0]["player"]["fullName"])
            blocker = curPlay["players"][0]["player"]["id"]
            fullName = curPlay["players"][0]["player"]["fullName"]
        elif(eventType == 'GOAL'):
            display(curPlay)
            
            for k in range(len(curPlay["players"])):
                player = curPlay['players'][k]['player']
                playerType = curPlay['players'][k]['playerType']
                print(player['fullName']  + " " + playerType)
            
        
        
        
        if(eventType == "BLOCKED_SHOT"):
            # Section for finding the index of a player
            addFlag = -1
            for k in range(len(gameArray)):
                if gameArray[k][0] == str(blocker):
                    addFlag = k
                    
            if addFlag == -1 :
                gameArray.append([str(blocker), fullName, 0, 0, 0, 0, 0, 1, 0.2])
            else:
                gameArray[addFlag][7] = gameArray[addFlag][7] + 1
                gameArray[addFlag][8] = gameArray[addFlag][8] + 0.2
    
    for j in range(len(gameArray)):
        dfSkater = dfSkater.append({'ID' : gameArray[j][0], 'Name' : gameArray[j][1], 'G' : gameArray[j][2], 'A' : gameArray[j][3], 'PIM' : gameArray[j][4], 'PPP' : gameArray[j][5], 'SHP' : gameArray[j][6], 'BLK' : gameArray[j][7], 'FPTS' : gameArray[j][8]}, ignore_index = True)
    #dfSkater = dfSkater.append({'Name' : fullName, 'ID' : str(blocker), 'G' : 0, 'A' : 0, 'PIM' : 0, 'PPP' : 0, 'SHP' : 0, 'BLK' : 1, 'FPTS' : 0.2}, ignore_index = True)
    
    display(gameArray)

display(dfSkater)
display(dfSkater.sort_values(by=['FPTS'], ascending=False))


    #goals
    
    #assists
    
    #blocks
    
    #pim
    
    #ppp
    
    #shp
    
    #w
    
    #ga
    
    #sv
    
    #s

,gamePk,link,gameType,season,gameDate,status.abstractGameState,status.codedGameState,status.detailedState,status.statusCode,status.startTimeTBD,...,teams.home.leagueRecord.ot,teams.home.leagueRecord.type,teams.home.score,teams.home.team.id,teams.home.team.name,teams.home.team.link,venue.name,venue.link,content.link,venue.id
0,2022020525,/api/v1/game/2022020525/feed/live,R,20222023,2022-12-22T19:00:00Z,Final,7,Final,7,False,...,6,league,4,10,Toronto Maple Leafs,/api/v1/teams/10,Scotiabank Arena,/api/v1/venues/null,/api/v1/game/2022020525/content,NaN
1,2022020526,/api/v1/game/2022020526/feed/live,R,20222023,2022-12-23T00:00:00Z,Final,7,Final,7,False,...,5,league,3,5,Pittsburgh Penguins,/api/v1/teams/5,PPG Paints Arena,/api/v1/venues/5034,/api/v1/game/2022020526/content,5034.0
2,2022020527,/api/v1/game/2022020527/feed/live,R,20222023,2022-12-23T00:00:00Z,Final,7,Final,7,False,...,5,league,5,3,New York Rangers,/api/v1/teams/3,Madison Square Garden,/api/v1/venues/5054,/api/v1/game/2022020527/content,5054.0
3,2022020528,/api/v1/game/2022020528/feed/live,R,20222023,2022-12-23T00:00:00Z,Final,7,Final,7,False,...,3,league,2,9,Ottawa Senators,/api/v1/teams/9,Canadian Tire Centre,/api/v1/venues/5031,/api/v1/game/2022020528/content,5031.0
4,2022020529,/api/v1/game/2022020529/feed/live,R,20222023,2022-12-23T00:00:00Z,Final,7,Final,7,False,...,2,league,3,6,Boston Bruins,/api/v1/teams/6,TD Garden,/api/v1/venues/5085,/api/v1/game/2022020529/content,5085.0
5,2022020530,/api/v1/game/2022020530/feed/live,R,20222023,2022-12-23T03:00:00Z,Final,7,Final,7,False,...,3,league,6,23,Vancouver Canucks,/api/v1/teams/23,Rogers Arena,/api/v1/venues/5073,/api/v1/game/2022020530/content,5073.0
6,2022020531,/api/v1/game/2022020531/feed/live,R,20222023,2022-12-23T03:30:00Z,Final,7,Final,7,False,...,5,league,4,26,Los Angeles Kings,/api/v1/teams/26,Crypto.com Arena,/api/v1/venues/null,/api/v1/game/2022020531/content,NaN
7,2022020532,/api/v1/game/2022020532/feed/live,R,20222023,2022-12-23T03:30:00Z,Final,7,Final,7,False,...,6,league,5,28,San Jose Sharks,/api/v1/teams/28,SAP Center at San Jose,/api/v1/venues/null,/api/v1/game/2022020532/content,NaN


/api/v1/game/2022020525/feed/live
GAME_SCHEDULED
PERIOD_READY
PERIOD_START
FACEOFF
SHOT
STOP
FACEOFF
BLOCKED_SHOT
MISSED_SHOT
GIVEAWAY
BLOCKED_SHOT
GIVEAWAY
HIT
HIT
GIVEAWAY
BLOCKED_SHOT
MISSED_SHOT
PENALTY
FACEOFF
SHOT
SHOT
HIT
GOAL


{'players': [{'player': {'id': 8477950,
    'fullName': 'Tony DeAngelo',
    'link': '/api/v1/people/8477950'},
   'playerType': 'Scorer',
   'seasonTotal': 5},
  {'player': {'id': 8475763,
    'fullName': 'Kevin Hayes',
    'link': '/api/v1/people/8475763'},
   'playerType': 'Assist',
   'seasonTotal': 21},
  {'player': {'id': 8474037,
    'fullName': 'James van Riemsdyk',
    'link': '/api/v1/people/8474037'},
   'playerType': 'Assist',
   'seasonTotal': 8},
  {'player': {'id': 8478492,
    'fullName': 'Ilya Samsonov',
    'link': '/api/v1/people/8478492'},
   'playerType': 'Goalie'}],
 'result': {'event': 'Goal',
  'eventCode': 'TOR16',
  'eventTypeId': 'GOAL',
  'description': 'Tony DeAngelo (5) Slap Shot, assists: Kevin Hayes (21), James van Riemsdyk (8)',
  'secondaryType': 'Slap Shot',
  'strength': {'code': 'PPG', 'name': 'Power Play'},
  'gameWinningGoal': False,
  'emptyNet': False},
 'about': {'eventIdx': 22,
  'eventId': 16,
  'period': 1,
  'periodType': 'REGULAR',
  'ordi

Tony DeAngelo Scorer
Kevin Hayes Assist
James van Riemsdyk Assist
Ilya Samsonov Goalie
FACEOFF
GIVEAWAY
SHOT
HIT
MISSED_SHOT
HIT
GIVEAWAY
HIT
STOP
FACEOFF
HIT
GIVEAWAY
HIT
SHOT
STOP
FACEOFF
HIT
HIT
BLOCKED_SHOT
HIT
HIT
HIT
HIT
HIT
TAKEAWAY
MISSED_SHOT
HIT
BLOCKED_SHOT
STOP
FACEOFF
HIT
HIT
SHOT
BLOCKED_SHOT
TAKEAWAY
BLOCKED_SHOT
STOP
FACEOFF
BLOCKED_SHOT
HIT
BLOCKED_SHOT
SHOT
STOP
FACEOFF
STOP
FACEOFF
BLOCKED_SHOT
SHOT
HIT
HIT
HIT
SHOT
STOP
FACEOFF
TAKEAWAY
GIVEAWAY
HIT
SHOT
HIT
SHOT
HIT
STOP
FACEOFF
HIT
HIT
BLOCKED_SHOT
BLOCKED_SHOT
PENALTY
FACEOFF
GIVEAWAY
MISSED_SHOT
SHOT
BLOCKED_SHOT
MISSED_SHOT
SHOT
BLOCKED_SHOT
TAKEAWAY
PERIOD_END
PERIOD_OFFICIAL
PERIOD_READY
PERIOD_START
FACEOFF
BLOCKED_SHOT
GIVEAWAY
SHOT
STOP
FACEOFF
HIT
HIT
SHOT
SHOT
SHOT
STOP
FACEOFF
TAKEAWAY
MISSED_SHOT
SHOT
MISSED_SHOT
SHOT
HIT
GIVEAWAY
MISSED_SHOT
SHOT
SHOT
STOP
FACEOFF
HIT
SHOT
TAKEAWAY
HIT
STOP
SHOT
PENALTY
PENALTY
FACEOFF
MISSED_SHOT
GIVEAWAY
SHOT
STOP
FACEOFF
TAKEAWAY
SHOT
STOP
FACEOFF
GIVEAWAY
MISSED_S

{'players': [{'player': {'id': 8475714,
    'fullName': 'Calle Jarnkrok',
    'link': '/api/v1/people/8475714'},
   'playerType': 'Scorer',
   'seasonTotal': 6},
  {'player': {'id': 8478483,
    'fullName': 'Mitchell Marner',
    'link': '/api/v1/people/8478483'},
   'playerType': 'Assist',
   'seasonTotal': 27},
  {'player': {'id': 8478115,
    'fullName': 'Pierre Engvall',
    'link': '/api/v1/people/8478115'},
   'playerType': 'Assist',
   'seasonTotal': 5},
  {'player': {'id': 8479394,
    'fullName': 'Carter Hart',
    'link': '/api/v1/people/8479394'},
   'playerType': 'Goalie'}],
 'result': {'event': 'Goal',
  'eventCode': 'TOR374',
  'eventTypeId': 'GOAL',
  'description': 'Calle Jarnkrok (6) Tip-In, assists: Mitchell Marner (27), Pierre Engvall (5)',
  'secondaryType': 'Tip-In',
  'strength': {'code': 'EVEN', 'name': 'Even'},
  'gameWinningGoal': False,
  'emptyNet': False},
 'about': {'eventIdx': 182,
  'eventId': 374,
  'period': 2,
  'periodType': 'REGULAR',
  'ordinalNum':

...

{'players': [{'player': {'id': 8478099,
    'fullName': 'Kevin Labanc',
    'link': '/api/v1/people/8478099'},
   'playerType': 'Scorer',
   'seasonTotal': 8}],
 'result': {'event': 'Goal',
  'eventCode': 'SJS685',
  'eventTypeId': 'GOAL',
  'description': 'Kevin Labanc (8) Wrist Shot, assists: none',
  'secondaryType': 'Wrist Shot',
  'strength': {'code': 'PPG', 'name': 'Power Play'},
  'gameWinningGoal': False,
  'emptyNet': True},
 'about': {'eventIdx': 295,
  'eventId': 685,
  'period': 3,
  'periodType': 'REGULAR',
  'ordinalNum': '3rd',
  'periodTime': '19:46',
  'periodTimeRemaining': '00:14',
  'dateTime': '2022-12-23T06:06:40Z',
  'goals': {'away': 2, 'home': 5}},
 'coordinates': {'x': 76.0, 'y': -38.0},
 'team': {'id': 28,
  'name': 'San Jose Sharks',
  'link': '/api/v1/teams/28',
  'triCode': 'SJS'}}

Kevin Labanc Scorer
FACEOFF
SHOT
PERIOD_END
PERIOD_OFFICIAL
GAME_END
GAME_OFFICIAL


[['8478136', 'Jake Middleton', 0, 0, 0, 0, 0, 4, 0.8],
 ['8481557', 'Matt Boldy', 0, 0, 0, 0, 0, 2, 0.4],
 ['8471274', 'Alex Goligoski', 0, 0, 0, 0, 0, 1, 0.2],
 ['8471709', 'Marc-Edouard Vlasic', 0, 0, 0, 0, 0, 1, 0.2],
 ['8476442', 'Matt Nieto', 0, 0, 0, 0, 0, 1, 0.2],
 ['8474009', 'Nick Bonino', 0, 0, 0, 0, 0, 2, 0.4],
 ['8478493', 'Joel Eriksson Ek', 0, 0, 0, 0, 0, 3, 0.6000000000000001],
 ['8476463', 'Jonas Brodin', 0, 0, 0, 0, 0, 4, 0.8],
 ['8480884', 'Calen Addison', 0, 0, 0, 0, 0, 1, 0.2],
 ['8478414', 'Timo Meier', 0, 0, 0, 0, 0, 1, 0.2],
 ['8477034', 'Jaycob Megna', 0, 0, 0, 0, 0, 1, 0.2],
 ['8474716', 'Jared Spurgeon', 0, 0, 0, 0, 0, 4, 0.8],
 ['8474053', 'Logan Couture', 0, 0, 0, 0, 0, 1, 0.2],
 ['8476449', 'Scott Harrington', 0, 0, 0, 0, 0, 2, 0.4],
 ['8475750', 'Jon Merrill', 0, 0, 0, 0, 0, 1, 0.2],
 ['8479983', 'Mario Ferraro', 0, 0, 0, 0, 0, 2, 0.4],
 ['8479981', 'Jonah Gadjovich', 0, 0, 0, 0, 0, 1, 0.2],
 ['8481477', 'Nico Sturm', 0, 0, 0, 0, 0, 1, 0.2],
 ['8476988', '

,Name,ID,G,A,PIM,PPP,SHP,BLK,FPTS
0,Owen Tippett,8480015,0,0,0,0,0,1,0.2
1,Jordie Benn,8474818,0,0,0,0,0,6,1.2
2,TJ Brodie,8474673,0,0,0,0,0,3,0.6
3,Cam York,8481546,0,0,0,0,0,1,0.2
4,Rasmus Ristolainen,8477499,0,0,0,0,0,3,0.6
...,...,...,...,...,...,...,...,...,...
140,Jon Merrill,8475750,0,0,0,0,0,1,0.2
141,Mario Ferraro,8479983,0,0,0,0,0,2,0.4
142,Jonah Gadjovich,8479981,0,0,0,0,0,1,0.2
143,Nico Sturm,8481477,0,0,0,0,0,1,0.2


,Name,ID,G,A,PIM,PPP,SHP,BLK,FPTS
66,Trevor van Riemsdyk,8477845,0,0,0,0,0,6,1.2
1,Jordie Benn,8474818,0,0,0,0,0,6,1.2
99,Jamie Oleksiak,8476467,0,0,0,0,0,6,1.2
126,Jake Middleton,8478136,0,0,0,0,0,4,0.8
20,Marcus Pettersson,8477969,0,0,0,0,0,4,0.8
...,...,...,...,...,...,...,...,...,...
51,Adam Fox,8479323,0,0,0,0,0,1,0.2
50,Jimmy Vesey,8476918,0,0,0,0,0,1,0.2
49,Brock Nelson,8475754,0,0,0,0,0,1,0.2
48,Barclay Goodrow,8476624,0,0,0,0,0,1,0.2
